<a href="https://colab.research.google.com/github/Growth-Hackers-SNU/sandbox-youtube-anlaysis/blob/main/code/deeplevel/KoBERT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT 적용
출처: https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb


In [ ]:
! pip install soynlp
! pip install kiwipiepy
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 419kB 24.2MB/s 
     |████████████████████████████████| 3.7MB 36.2MB/s 
     |████████████████████████████████| 57.3MB 47kB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.9.0-cp37-none-any.whl size=57496291 sha256=0906fb599ea09066986930ba9bf96e39519055eccbb26dc2c10cc728f018ab91
  Stored in directory: /root/.cache/pip/wheels/61/19/a2/f85a7031e0ff829231b4562e6460a84fef599080bbb2573ef7
Successfully built kiwipiepy-model
     |████████████████████████████████| 46.9MB 62kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 35.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595709 sha256=667f09a068fba075c4a3243cd61fb13cd5f0e47a9353fe5db8b8b47974a159bc
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Succes

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-q0u1qvcd
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-q0u1qvcd
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12718 sha256=29ecd6d96f8dac3d7c5faf13078c40e6c5f763817431f03175fb2b342baa2a88
  Stored in directory: /tmp/pip-ephem-wheel-cache-kf6s_ybe/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm, tqdm_notebook
import pickle 

import matplotlib.pyplot as plt

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading KoBERT

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


# Load DataSet

In [ ]:
data_DIR = '/content/drive/Shareddrives/[GH x Sandbox]/최종분류/sample_final.csv'
raw_df = pd.read_csv(data_DIR)

In [ ]:
data_df = raw_df[['comment', 'class']]
data_df.head()

,comment,class
0,흠.. 일단 전 안맞을래요,7
1,자식새끼 키워봤자 아무소용업다..,7
2,방테나ㅋㅋㅋ,2
3,하람님 저고민있어욧!!\n노래잘부르고싶은데..어떻게하면 잘 부를수있을까요?오늘도 영...,1
4,희재도 준비하고있고 발표예정이예요\n다시 확인해주세요 모두 준비하고있어요 계약때문에...,7


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
encoder = LabelEncoder()
encoder.fit(data_df['class'])
data_df['class'] = encoder.transform(data_df['class'])
data_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,comment,class
0,흠.. 일단 전 안맞을래요,6
1,자식새끼 키워봤자 아무소용업다..,6
2,방테나ㅋㅋㅋ,1
3,하람님 저고민있어욧!!\n노래잘부르고싶은데..어떻게하면 잘 부를수있을까요?오늘도 영...,0
4,희재도 준비하고있고 발표예정이예요\n다시 확인해주세요 모두 준비하고있어요 계약때문에...,6


In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7}

In [ ]:
train, test = train_test_split(data_df, test_size=0.2, random_state=7607) 
print("train shape is:", len(train)) 
print("test shape is:", len(test))

train shape is: 17600
test shape is: 4400


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        if isinstance(dataset, pd.core.frame.DataFrame):
            sentences = []
            labels = []
            for i, row in dataset.iterrows():
                sentence = row[sent_idx]
                sentence = re.sub(r'[0-9]+', '', str(sentence))
                sentences.append(transform(sentence))
                label = row[label_idx]
                labels.append(np.int32(label))
            self.sentences = sentences
            self.labels = labels
        else:
            self.sentences = [transform([i[sent_idx]]) for i in dataset]
            self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 32
batch_size = 8
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2,  517, 7948,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
       dtype=int32),
 array(4, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 6)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# Modeling

In [ ]:
# 출처: https://moondol-ai.tistory.com/241
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), 
                              attention_mask=attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 1.814042329788208 train acc 0.25
epoch 1 batch id 201 loss 1.3220285177230835 train acc 0.43594527363184077
epoch 1 batch id 401 loss 1.1571937799453735 train acc 0.49345386533665836
epoch 1 batch id 601 loss 1.0184859037399292 train acc 0.5014559068219634
epoch 1 batch id 801 loss 2.014948844909668 train acc 0.5081148564294632
epoch 1 batch id 1001 loss 1.2803348302841187 train acc 0.5092407592407593
epoch 1 batch id 1201 loss 1.4114083051681519 train acc 0.5104079933388843
epoch 1 batch id 1401 loss 1.5632812976837158 train acc 0.5137401855817273
epoch 1 batch id 1601 loss 1.5194628238677979 train acc 0.5178013741411618
epoch 1 batch id 1801 loss 1.3777997493743896 train acc 0.52005830094392
epoch 1 batch id 2001 loss 1.603108525276184 train acc 0.5223013493253373

epoch 1 train acc 0.5232386363636363


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.5288636363636363


epoch 2 batch id 1 loss 0.9464792013168335 train acc 0.75
epoch 2 batch id 201 loss 1.3099970817565918 train acc 0.5317164179104478
epoch 2 batch id 401 loss 1.1621038913726807 train acc 0.5420822942643392
epoch 2 batch id 601 loss 1.014151692390442 train acc 0.5349417637271214
epoch 2 batch id 801 loss 2.216909170150757 train acc 0.5337078651685393
epoch 2 batch id 1001 loss 1.2387065887451172 train acc 0.5295954045954046
epoch 2 batch id 1201 loss 1.309017539024353 train acc 0.5283097418817652
epoch 2 batch id 1401 loss 1.3658937215805054 train acc 0.5294432548179872
epoch 2 batch id 1601 loss 1.3716106414794922 train acc 0.5316989381636478
epoch 2 batch id 1801 loss 1.373063087463379 train acc 0.5325513603553581
epoch 2 batch id 2001 loss 1.5989493131637573 train acc 0.533608195902049

epoch 2 train acc 0.5335795454545454


...


epoch 3 test acc 0.5288636363636363


epoch 4 batch id 1 loss 0.936526358127594 train acc 0.75
epoch 4 batch id 201 loss 1.2326154708862305 train acc 0.5310945273631841
epoch 4 batch id 401 loss 1.0491431951522827 train acc 0.5414588528678305
epoch 4 batch id 601 loss 0.9858729839324951 train acc 0.5345257903494176
epoch 4 batch id 801 loss 2.1115975379943848 train acc 0.5333957553058677
epoch 4 batch id 1001 loss 1.3396679162979126 train acc 0.5293456543456544
epoch 4 batch id 1201 loss 1.2638322114944458 train acc 0.5284138218151541
epoch 4 batch id 1401 loss 1.382439374923706 train acc 0.5295324768022841
epoch 4 batch id 1601 loss 1.3667998313903809 train acc 0.5317770143660212
epoch 4 batch id 1801 loss 1.4786056280136108 train acc 0.5326207662409772
epoch 4 batch id 2001 loss 1.6319935321807861 train acc 0.5336706646676662

epoch 4 train acc 0.5336363636363637



epoch 4 test acc 0.5288636363636363


epoch 5 batch id 1 loss 0.8632665276527405 train acc 0.75
epoch 5 batch id 201 loss 1.2584401369094849 train acc 0.5304726368159204
epoch 5 batch id 401 loss 0.9732133746147156 train acc 0.5417705735660848
epoch 5 batch id 601 loss 0.9864968657493591 train acc 0.535981697171381
epoch 5 batch id 801 loss 1.995226263999939 train acc 0.535892634207241
epoch 5 batch id 1001 loss 1.5488053560256958 train acc 0.532967032967033
epoch 5 batch id 1201 loss 1.3017849922180176 train acc 0.5315362198168193
epoch 5 batch id 1401 loss 1.1953704357147217 train acc 0.5330121341898644
epoch 5 batch id 1601 loss 1.239241600036621 train acc 0.5354465958775765
epoch 5 batch id 1801 loss 1.4027360677719116 train acc 0.5365769017212659
epoch 5 batch id 2001 loss 1.58836829662323 train acc 0.5374812593703149

epoch 5 train acc 0.537784090909091



epoch 5 test acc 0.5306818181818181
